In [ ]:
#works with cv virtual environment

In [1]:
import sys
import glob
import os
import cv2
import numpy as np

from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2D
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras import backend as K
import keras

Using Theano backend.
WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named mkl


In [2]:
folder_path = '/home/niaki/Code/hpatches/hpatches-benchmark/data/hpatches-release'
patch_size = 65

In [3]:
# all types of patches 
tps = ['ref','e1','e2','e3','e4','e5','h1','h2','h3','h4','h5',\
       't1','t2','t3','t4','t5']

class hpatches_sequence:
    """Class for loading an HPatches sequence from a sequence folder"""
    itr = tps
    def __init__(self,base):
        name = base.split('/')
        self.name = name[-1]
        self.base = base
        for t in self.itr:
            im_path = os.path.join(base, t+'.png')
            im = cv2.imread(im_path,0)
            self.N = im.shape[0]/patch_size
            setattr(self, t, np.split(im, self.N))

In [4]:
seqs = glob.glob(folder_path+'/*')
seqs = [os.path.abspath(p) for p in seqs]

In [5]:
len(seqs)

116

In [6]:
descr_name = 'ae_32'
base_dir = '/home/niaki/PycharmProjects/patch-desc-ae/models'
model_version = '32'

In [7]:
encoder_3conv3mp = load_model(base_dir + '/encoder' + model_version + '.h5')

In [8]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, patch_size, patch_size)
else:
    input_shape = (patch_size, patch_size, 3)

input_img = Input(shape=input_shape)

x = Conv2D(16, (3, 3), activation="relu", padding="same")(input_img)
x = Conv2D(8, (3, 3), activation="relu", padding="same")(x)
x = Conv2D(8, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

encoder = Model(input_img, encoded)

for i in range(len(encoder_3conv3mp.layers)):
    encoder.get_layer(index=i).set_weights(encoder_3conv3mp.get_layer(index=i).get_weights())


In [9]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 65, 65, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 65, 65, 16)        448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 65, 65, 8)         1160      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 65, 65, 8)         584       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 33, 33, 8)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 8)         0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 9, 8)           0         
Total para

In [10]:
# output shape, and flattened shape
print(encoder.output_shape[1:])
print(np.prod(encoder.output_shape[1:]))

(9, 9, 8)
648


In [ ]:
# for seq_path in seqs:
#     seq = hpatches_sequence(seq_path)
#     path = os.path.join(descr_name,seq.name)
#     if not os.path.exists(path):
#         os.makedirs(path)
#     descr = np.zeros((seq.N,2)) # trivial (mi,sigma) descriptor
#     for tp in tps:
#         print(seq.name+'/'+tp)
#         for i,patch in enumerate(getattr(seq, tp)):
#             mi = np.mean(patch) # trivial (mi,sigma) descriptor
#             sigma = np.std(patch) # trivial (mi,sigma) descriptor
#             descr[i] = np.array([mi,sigma])
#         np.savetxt(os.path.join(path,tp+'.csv'), descr, delimiter=',') # X is an array

In [12]:
for seq_path in seqs:
    seq = hpatches_sequence(seq_path)
    path = os.path.join(descr_name,seq.name)
    if not os.path.exists(path):
        os.makedirs(path)

    descr_size_per_patch = np.prod(encoder.output_shape[1:])
    descr = np.zeros((int(seq.N),descr_size_per_patch)) # trivial (mi,sigma) descriptor   
    
    for tp in tps:
        print(seq.name+'/'+tp)
        for i,patch in enumerate(getattr(seq, tp)):
            patch = patch / 255.0
            #print(patch)
            patch3D = np.repeat(patch, 3, axis=1).reshape((patch_size, patch_size, 3))
            patch3D_encoded = encoder.predict(np.expand_dims(patch3D, axis=0))
            patch3D_encoded_flat = patch3D_encoded.flatten()
            patch3D_encoded_flat_rounded = patch3D_encoded_flat.round(decimals=8)
#             print(patch3D_encoded_flat_rounded.shape)
#             print(type(patch3D_encoded_flat_rounded[0]))
#             print(patch3D_encoded_flat_rounded)
#             print()
            descr[i] = patch3D_encoded_flat_rounded
        np.savetxt(os.path.join(path,tp+'.csv'), descr, delimiter=',') # X is an array

i_kurhaus/ref
i_kurhaus/e1
i_kurhaus/e2
i_kurhaus/e3
i_kurhaus/e4
i_kurhaus/e5
i_kurhaus/h1
i_kurhaus/h2
i_kurhaus/h3
i_kurhaus/h4
i_kurhaus/h5
i_kurhaus/t1
i_kurhaus/t2
i_kurhaus/t3
i_kurhaus/t4
i_kurhaus/t5
i_veggies/ref
i_veggies/e1
i_veggies/e2
i_veggies/e3
i_veggies/e4
i_veggies/e5
i_veggies/h1
i_veggies/h2
i_veggies/h3
i_veggies/h4
i_veggies/h5
i_veggies/t1
i_veggies/t2
i_veggies/t3
i_veggies/t4
i_veggies/t5
i_fruits/ref
i_fruits/e1
i_fruits/e2
i_fruits/e3
i_fruits/e4
i_fruits/e5
i_fruits/h1
i_fruits/h2
i_fruits/h3
i_fruits/h4
i_fruits/h5
i_fruits/t1
i_fruits/t2
i_fruits/t3
i_fruits/t4
i_fruits/t5
v_grace/ref
v_grace/e1
v_grace/e2
v_grace/e3
v_grace/e4
v_grace/e5
v_grace/h1
v_grace/h2
v_grace/h3
v_grace/h4
v_grace/h5
v_grace/t1
v_grace/t2
v_grace/t3
v_grace/t4
v_grace/t5
v_bark/ref
v_bark/e1
v_bark/e2
v_bark/e3
v_bark/e4
v_bark/e5
v_bark/h1
v_bark/h2
v_bark/h3
v_bark/h4
v_bark/h5
v_bark/t1
v_bark/t2
v_bark/t3
v_bark/t4
v_bark/t5
i_boutique/ref
i_boutique/e1
i_boutique/e2
i_boutiq

KeyboardInterrupt: 